In [39]:
!pip install nba-on-court


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [40]:
from nba_api.stats.endpoints import playbyplayv2
import nba_on_court.nba_on_court as noc
import pandas as pd
import numpy as np

### Let's download pbp data from 2022/23 NBA Playoffs season nba.stats
noc.load_nba_data(seasons=2022, data='nbastats', seasontype='po', untar=True)

In [41]:
po_df = pd.read_csv('nbastats_po_2022.csv')
po_df.loc[po_df['GAME_ID'] == 42200101]
mil_mia = noc.players_on_court(po_df.loc[po_df['GAME_ID'] == 42200101])
subset = pd.concat([mil_mia.loc[:,['GAME_ID', 'SCOREMARGIN', 'SCORE']], mil_mia.iloc[:,34:]])
subset[subset['SCOREMARGIN'].isna() == False]

,GAME_ID,SCOREMARGIN,SCORE,AWAY_PLAYER1,AWAY_PLAYER2,AWAY_PLAYER3,AWAY_PLAYER4,AWAY_PLAYER5,HOME_PLAYER1,HOME_PLAYER2,HOME_PLAYER3,HOME_PLAYER4,HOME_PLAYER5
4,42200101.0,2,0 - 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,42200101.0,TIE,2 - 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,42200101.0,2,2 - 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,42200101.0,-1,5 - 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,42200101.0,-3,7 - 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,42200101.0,-8,125 - 117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,42200101.0,-11,128 - 117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464,42200101.0,-12,129 - 117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,42200101.0,-13,130 - 117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
pbp = playbyplayv2.PlayByPlayV2(game_id="0022100001").play_by_play.get_data_frame()
pbp_with_players = noc.players_on_court(pbp)
len(pbp_with_players.columns) - len(pbp.columns)
pbp_with_players.columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG', 'AWAY_PLAYER1', 'AWAY_PLAYER2', 'AWAY_PLAYER3',
       'AWAY_PLAYER4', 'AWAY_PLAYER5', 'HOME_PLAYER1', 'HOME_PLAYER2',
       'HOME_PLAYER3', 'HOME_PLAYER4', 'HOME_PLAYER5'],
      dtype='object')

In [203]:
subset = pbp_with_players.loc[:,['SCORE', 'SCOREMARGIN', 'AWAY_PLAYER1', 'AWAY_PLAYER2', 'AWAY_PLAYER3', 'AWAY_PLAYER4', 'AWAY_PLAYER5', 'HOME_PLAYER1', 'HOME_PLAYER2', 'HOME_PLAYER3', 'HOME_PLAYER4', 'HOME_PLAYER5']]
subset = subset[subset['SCOREMARGIN'].isna() == False].replace('TIE', 0)
subset['PM'] = subset['SCOREMARGIN'].astype(int).diff().replace(np.nan, 0)
subset = subset.reset_index(drop = True)
subset

,SCORE,SCOREMARGIN,AWAY_PLAYER1,AWAY_PLAYER2,AWAY_PLAYER3,AWAY_PLAYER4,AWAY_PLAYER5,HOME_PLAYER1,HOME_PLAYER2,HOME_PLAYER3,HOME_PLAYER4,HOME_PLAYER5,PM
0,2 - 0,-2,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,0.0
1,2 - 2,0,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,2.0
2,2 - 5,3,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,3.0
3,4 - 5,1,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,-2.0
4,4 - 7,3,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,102 - 125,23,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,3.0
117,103 - 125,22,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,-1.0
118,104 - 125,21,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,-1.0
119,104 - 127,23,1628971,1627761,201949,1628975,1630560,203648,1630572,1630686,1629670,1629620,2.0


In [209]:
subset['SUB']= False

for i in range(subset.shape[0]-1): 
    subset.loc[i+1,'SUB'] = list(subset.iloc[i+1,2:12]) != list(subset.iloc[i,2:12])

In [210]:
subset

,SCORE,SCOREMARGIN,AWAY_PLAYER1,AWAY_PLAYER2,AWAY_PLAYER3,AWAY_PLAYER4,AWAY_PLAYER5,HOME_PLAYER1,HOME_PLAYER2,HOME_PLAYER3,HOME_PLAYER4,HOME_PLAYER5,PM,SUB
0,2 - 0,-2,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,0.0,False
1,2 - 2,0,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,2.0,False
2,2 - 5,3,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,3.0,False
3,4 - 5,1,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,-2.0,False
4,4 - 7,3,201142,1629651,201933,201935,203925,201572,201950,1628960,203114,203507,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,102 - 125,23,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,3.0,False
117,103 - 125,22,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,-1.0,False
118,104 - 125,21,1628971,1627761,201949,1628975,1630560,203648,1630572,1626192,1629670,1629620,-1.0,False
119,104 - 127,23,1628971,1627761,201949,1628975,1630560,203648,1630572,1630686,1629670,1629620,2.0,True


In [6]:
players_id = list(pbp_with_players.iloc[0, 34:].reset_index(drop=True))
print(players_id)

[201142, 1629651, 201933, 201935, 203925, 201572, 201950, 1628960, 203114, 203507]


In [7]:
players_name = noc.players_name(players_id)
print(players_name)

['Kevin Durant', 'Nic Claxton', 'Blake Griffin', 'James Harden', 'Joe Harris', 'Brook Lopez', 'Jrue Holiday', 'Grayson Allen', 'Khris Middleton', 'Giannis Antetokounmpo']


### Loading in Combined Dataset

In [42]:
noc.load_nba_data(seasons=2022, data=('nbastats', 'pbpstats'), seasontype='po', untar=True)

nbastats = pd.read_csv('nbastats_po_2022.csv')
pbpstats = pd.read_csv('pbpstats_po_2022.csv')

nbastats = nbastats.loc[nbastats['GAME_ID'] == 42200405].reset_index(drop=True) 
pbpstats = pbpstats.loc[pbpstats['GAMEID'] == 42200405].reset_index(drop=True) 

print(nbastats.shape, pbpstats.shape)
((463, 34), (396, 19))

full_pbp = noc.left_join_nbastats(nbastats, pbpstats)
print(full_pbp.shape)
(463, 50)

(463, 34) (396, 19)
(463, 50)


(463, 50)

In [60]:
full_pbp[(full_pbp['SCOREMARGIN'].isna() == False)].columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG', 'DESCRIPTION_STATS', 'ENDTIME', 'EVENTS',
       'FG2A', 'FG2M', 'FG3A', 'FG3M', 'GAMEDATE', 'GAMEID',
       'NONSHOOTINGFOULSTHATRESULTEDINFTS', 'OFFENSIVEREBOUNDS', 'OPPONENT',
       'SHOOTINGFOULSDRAWN', 'STARTSCOREDIFFERENTIAL', 'STARTTIME',
       'STARTTYPE', 'TURNOVERS', 'DESCRIPTION_PBP', 'URL'],
      dtype='object')

In [1]:
full_pbp.columns 

NameError: name 'full_pbp' is not defined